In [12]:
! pip install huggingface_hub 
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.3 MB/s eta 0:00:00


# get data

In [2]:
from datasets import load_dataset
data = load_dataset("squad")
data

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

# metric

In [3]:
from datasets import load_metric
metric = load_metric("squad")
def compute_metrics(eval_pred):
    logits , labels = eval_pred
    predictions = np.argmax(logits,axis=1)
    return metric.compute(predictions=predictions,references=labels)


<ipython-input-3-84aa221b625f>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


# get model and tokenizer

In [4]:
from transformers import AutoTokenizer, BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# process data

In [5]:
def process(data):
  from datasets import Dataset
  dict_response = {
    "context":[],
    "questions":[],
    "text":[],
    "start_positions":[],
    "end_positions":[]
  }
  for i in data:
    dict_response["text"].append(i["answers"]["text"][0])
    dict_response["start_positions"].append(i["answers"]["answer_start"][0])
    dict_response["end_positions"].append(i["answers"]["answer_start"][0]+len(i["answers"]["text"][0]))
    dict_response["context"].append(i["context"])
    dict_response["questions"].append(i["question"])
  data_tokenizer = tokenizer(dict_response["questions"],dict_response["context"],truncation=True,text_target=dict_response["text"])
  dict_tok = {
    "input_ids":data_tokenizer["input_ids"],
    "token_type_ids":data_tokenizer["token_type_ids"],
    "attention_mask":data_tokenizer["attention_mask"],
    "labels":data_tokenizer["labels"],
    "start_positions":dict_response["start_positions"],
    "end_positions":dict_response["end_positions"]
  }
  dataset = Dataset.from_dict(dict_tok)
  del dict_tok
  del data_tokenizer
  del dict_response
  return dataset

In [6]:
data_train = process(data["train"])
data_val = process(data["validation"])

In [7]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,DataCollatorForSeq2Seq
collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [8]:

from huggingface_hub import notebook_login

notebook_login()
     

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
args = Seq2SeqTrainingArguments(
    'test7',
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    log_level="error",
    predict_with_generate=True,
    num_train_epochs=10,
    push_to_hub = True,
    optim="adamw_torch"
    # hub_model_id='organization/',
    # push_to_hub_token=token_login
)


In [12]:
train = Seq2SeqTrainer(
    model,
    args,
    train_dataset = data_train,
    eval_dataset = data_val,
    data_collator = collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)


Cloning https://huggingface.co/Ahmade/test7 into local empty directory.


In [ ]:
train.train()

In [ ]:
train.push_to_hub("hello")
